In [1]:
%load_ext aiida
%aiida

Loaded AiiDA DB environment - profile name: presto.

In [ ]:
from xtalpaint.aiida.workgraphs import setup_inpainting_wg
from aiida.orm import StructureData
from aiida_workgraph.orm.workgraph import WorkGraphNode

from aiida_workgraph import WorkGraph, task
import yaml
from xtalpaint.inpainting.config_schema import InpaintingWorkGraphConfig
from xtalpaint.aiida.data import BatchedStructures, BatchedStructuresData
from xtalpaint.utils.relaxation_utils import relax_structures

MODELS_PROJECT_ROOT: /home/reents_t/project/dev-mattergen-inpainting/git/mattergen/mattergen


In [3]:
def load_mc3d_h_structures(label):
    query_structures = QueryBuilder().append(
        Group, filters={'label': f'MC3D-with-H/{label}'},
        tag='group'
    ).append(
        StructureData, tag='structure', with_group='group'
    )

    structures = query_structures.all(flat=True)
    return structures

In [4]:
query_structures = QueryBuilder().append(
    # Group, filters={'label': 'test/structures'},
    Group, filters={'label': 'MC3D-with-H-relaxed'},
    tag='group'
).append(
    StructureData, tag='structure', with_group='group'
).limit(10)

structures = query_structures.all(flat=True)
structures = [
    # s for s in structures if 20 < len(s.sites) <= 40
    s for s in structures if len(s.sites) <= 10
    ]

In [5]:
len(structures)

5

In [ ]:
param_grid = {
    "N_steps": 5,
    "coordinates_snr": 0.2,
    "n_corrector_steps": 1,
    "batch_size": 1000,
}

{'name': 'TD-1-samples',
 'fix_cell': True,
 'max_num_atoms': 20,
 'predictor_corrector': 'new-timesteps',
 'param_grid': {'N_steps': 100,
  'coordinates_snr': 0.2,
  'n_corrector_steps': 1,
  'batch_size': 1000,
  'N_samples_per_structure': 1},
 'relax': True,
 'relax_kwargs': {'load_path': 'MatterSim-v1.0.0-5M.pth', 'fmax': 0.0025},
 'model_path': '/data/user/reents_t/projects/mlip/git/mattergen/checkpoints/new-td-20-perc',
 'save_prefix': 'TD-1-samples'}

In [ ]:
ENV_ACTIVATION_CMD = "source ~/.aiida_venvs/dev-mattergen-inpainting/bin/activate"

inputs = InpaintingWorkGraphConfig(
    inpainting_pipeline_params={
        "predictor_corrector": "baseline",
        "inpainting_model_params": param_grid,
        "pretrained_name": "mattergen_base",
        "sampling_config_path": "/home/reents_t/project/dev-mattergen-inpainting/git/mattergen/sampling_conf",
    },
    structures=BatchedStructures(
        {s.uuid.replace("-", "_"): s.get_pymatgen() for s in structures}
    ),
    gen_inpainting_candidates_params={
        "n_inp": {
            s.uuid.replace("-", "_"): s.get_site_kindnames().count("H")
            for s in structures
        },
        "element": "H",
        "num_samples": 1,
    },
    relax=True,
    full_relax=True,
    full_relax_wo_pre_relax=False,
    relax_kwargs={
        "elements_to_relax": ["H"],
        "fmax": 0.01,
        "max_natoms_per_batch": 5000,
        "load_path": "MatterSim-v1.0.0-5M.pth",
        "max_n_steps": 50,
        "device": "cuda",
        "mlip": "mattersim",
        "optimizer": "BFGS",
        "return_initial_energies": False,
        "return_initial_forces": False,
        "return_final_forces": False,
    },
    gen_inpainting_candidates_options={
        "custom_scheduler_commands": f"{ENV_ACTIVATION_CMD}\nexport PYTHONBREAKPOINT=0",
    },
    options={
        "prepend_text": f"{ENV_ACTIVATION_CMD}\nexport PYTHONBREAKPOINT=0",
    },
    evaluate_params={"max_workers": 5, "metrics": ["match", "rmsd"]},
    evaluate=True,
)

In [ ]:
wg = setup_inpainting_wg(
    inputs
)

In [ ]:
wg

In [ ]:
wg.run()

12/12/2025 08:59:33 AM <1333238> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [231908|WorkGraphEngine|continue_workgraph]: tasks ready to run: generate_inpainting_candidates
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[231908|WorkGraphEngine|continue_workgraph]: tasks ready to run: generate_inpainting_candidates


Input ports: {'name': 'inputs', 'identifier': 'workgraph.namespace', 'required': False, 'ports': [{'name': 'element', 'identifier': 'workgraph.any', 'required': True}, {'name': 'n_inp', 'identifier': 'workgraph.any', 'required': True}, {'name': 'num_samples', 'identifier': 'workgraph.int', 'required': False}, {'name': 'structures', 'identifier': 'workgraph.any', 'required': False}]}
function_inputs: {'element': <Str: uuid: 3dc989ce-498d-475c-8250-2d5a1d5dd292 (pk: 231886) value: H>, 'n_inp': <Dict: uuid: 91af5349-8391-4658-b57c-67a575316b4c (pk: 231885)>, 'num_samples': <Int: uuid: eb6f1866-d023-466a-a05d-4ec540290afc (pk: 231887) value: 5>, 'structures': <BatchedStructuresData: uuid: e48ed4c5-2148-4287-9a2a-aead80d37cbf (pk: 231884)>}


12/12/2025 08:59:35 AM <1333238> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [231908|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 231914
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[231908|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 231914
12/12/2025 08:59:38 AM <1333238> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [231908|WorkGraphEngine|update_task_state]: Task: generate_inpainting_candidates, type: PythonJob, finished.
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[231908|WorkGraphEngine|update_task_state]: Task: generate_inpainting_candidates, type: PythonJob, finished.
12/12/2025 08:59:40 AM <1333238> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [231908|WorkGraphEngine|continue_workgraph]: tasks ready to run: inpainting
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[231908|WorkGraphEngine|continue_workgraph

Input ports: {'name': 'inputs', 'identifier': 'workgraph.namespace', 'required': False, 'ports': [{'name': 'config', 'identifier': 'workgraph.any', 'required': True}, {'name': 'structures', 'identifier': 'workgraph.any', 'required': False}, {'name': 'usempi', 'identifier': 'workgraph.bool', 'required': False}]}
function_inputs: {'config': <Dict: uuid: 10d156f5-80e7-4c96-9e8c-811a1ae4c926 (pk: 231888)>, 'structures': <BatchedStructuresData: uuid: 21eb22c2-0ad5-4d8a-b84e-d008ae6d369e (pk: 231917)>, 'usempi': <Bool: uuid: 6db6afb0-b961-49c1-9d7d-38ed222e0e43 (pk: 231889) value: False>}


12/12/2025 08:59:42 AM <1333238> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [231908|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 231923
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[231908|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 231923
12/12/2025 08:59:45 AM <1333238> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [231908|WorkGraphEngine|update_task_state]: Task: inpainting, type: PythonJob, finished.
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[231908|WorkGraphEngine|update_task_state]: Task: inpainting, type: PythonJob, finished.
12/12/2025 08:59:47 AM <1333238> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [231908|WorkGraphEngine|continue_workgraph]: tasks ready to run: inpainted_constrained_relaxation,evaluate_inpainting_rmsd_inpainting,evaluate_inpainting_match_inpainting
REPORT:aiida.orm.nodes.process.workflow.workchain.W

...

12/12/2025 09:00:21 AM <1333238> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [231908|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 231955, 231959
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[231908|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 231955, 231959
/home/reents_t/project/dev-mattergen-inpainting/git/dbcsi-cleanup/src/dbcsi_inpainting/aiida/data.py:315: AiidaDeprecationWarning: `PandasDataFrameData.attributes` is deprecated, use `PandasDataFrameData.base.attributes.all` instead. (this will be removed in v3)
  handle, self.attributes.get("filename")


[{'name': 'metric_results', 'required': False, 'identifier': 'workgraph.any', 'value': <PandasDataFrameData: uuid: c4dd6a5a-0516-4e7c-b33c-1609d07328c6 (unstored)>}]


/home/reents_t/project/dev-mattergen-inpainting/git/dbcsi-cleanup/src/dbcsi_inpainting/aiida/data.py:315: AiidaDeprecationWarning: `PandasDataFrameData.attributes` is deprecated, use `PandasDataFrameData.base.attributes.all` instead. (this will be removed in v3)
  handle, self.attributes.get("filename")


[{'name': 'metric_results', 'required': False, 'identifier': 'workgraph.any', 'value': <PandasDataFrameData: uuid: b8f5f51c-7f08-480a-8a2a-79576521b63b (unstored)>}]


12/12/2025 09:00:33 AM <1333238> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [231908|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[231908|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
12/12/2025 09:00:34 AM <1333238> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [231908|WorkGraphEngine|finalize]: Finalize workgraph.
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[231908|WorkGraphEngine|finalize]: Finalize workgraph.
